In [1]:
from pitcheranalyze import pitcheranalyze
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [2]:


pre_make_pivot_table_df = pd.read_excel("../data/pre_make_pivot_table_draft5.xlsx", dtype = {"gameId":"str"})
pd.set_option("display.max_columns", None)

In [3]:
teamdata_df = pre_make_pivot_table_df[["gameId", "side", "teamWinrate", "teamGoldDiff", "teamKillDiff"]].drop_duplicates()

teamdata_df = teamdata_df.pivot(index='gameId', columns='side', values=['teamWinrate', "teamGoldDiff", "teamKillDiff"])
# Flatten the multi-level columns and adjust the column names in one step
teamdata_df.columns = [f"{col[1]}_{col[0]}".replace('team', '').replace('__', '_').strip() for col in teamdata_df.columns.values]
teamdata_df

,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff
gameId,,,,,,
107405837336900396,0.565217,0.214286,0.208655,-3.059317,0.000120,-0.002194
107405837336900398,0.272727,0.520000,-2.828609,-0.020675,-0.002835,-0.000087
107405837336900400,0.500000,0.518519,-0.298304,0.266528,-0.000239,0.000405
107405837336900402,0.583333,0.541667,0.719165,0.063835,0.000336,-0.000091
107405837336900404,0.300000,0.590909,-2.337897,0.528457,-0.001861,0.000453
...,...,...,...,...,...,...
112439909523296517,0.312500,0.588235,-2.531671,0.806513,-0.002313,0.000887
112439909523296534,0.585366,0.370370,1.482972,-2.140650,0.000873,-0.002502
112439909523296535,0.357143,0.595238,-2.673311,1.845644,-0.003277,0.001417


In [4]:
headtohead_and_winner_df = pre_make_pivot_table_df[["gameId", "headtoHeadWinrate", "headtoHeadGoldDiff", "headtoHeadKillDiff", "winner"]].drop_duplicates()

In [5]:
formvalue_pivot = pd.pivot_table(
    pre_make_pivot_table_df,
    values = "formvalue",
    index = ["gameId"],
    columns = ["side", "role","elements"]
).reset_index()
formvalue_pivot.columns = ['_'.join(col) if isinstance(col, tuple) else col for col in formvalue_pivot.columns]
formvalue_pivot.rename(columns = {"gameId__" : "gameId"}, inplace = True)
formvalue_pivot.head()

,gameId,Blue_Adc_assistsPerTime,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_deathsPerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_killsPerTime,Blue_Adc_wardsScorePerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,Blue_Jgl_killsPerTime,Blue_Jgl_wardsScorePerTime,Blue_Mid_assistsPerTime,Blue_Mid_championDamageShare,Blue_Mid_creepScorePerTime,Blue_Mid_deathsPerTime,Blue_Mid_goldEarnedPerTime,Blue_Mid_kda,Blue_Mid_killsPerTime,Blue_Mid_wardsScorePerTime,Blue_Spt_assistsPerTime,Blue_Spt_championDamageShare,Blue_Spt_creepScorePerTime,Blue_Spt_deathsPerTime,Blue_Spt_goldEarnedPerTime,Blue_Spt_kda,Blue_Spt_killsPerTime,Blue_Spt_wardsScorePerTime,Blue_Top_assistsPerTime,Blue_Top_championDamageShare,Blue_Top_creepScorePerTime,Blue_Top_deathsPerTime,Blue_Top_goldEarnedPerTime,Blue_Top_kda,Blue_Top_killsPerTime,Blue_Top_wardsScorePerTime,Red_Adc_assistsPerTime,Red_Adc_championDamageShare,Red_Adc_creepScorePerTime,Red_Adc_deathsPerTime,Red_Adc_goldEarnedPerTime,Red_Adc_kda,Red_Adc_killsPerTime,Red_Adc_wardsScorePerTime,Red_Jgl_assistsPerTime,Red_Jgl_championDamageShare,Red_Jgl_creepScorePerTime,Red_Jgl_deathsPerTime,Red_Jgl_goldEarnedPerTime,Red_Jgl_kda,Red_Jgl_killsPerTime,Red_Jgl_wardsScorePerTime,Red_Mid_assistsPerTime,Red_Mid_championDamageShare,Red_Mid_creepScorePerTime,Red_Mid_deathsPerTime,Red_Mid_goldEarnedPerTime,Red_Mid_kda,Red_Mid_killsPerTime,Red_Mid_wardsScorePerTime,Red_Spt_assistsPerTime,Red_Spt_championDamageShare,Red_Spt_creepScorePerTime,Red_Spt_deathsPerTime,Red_Spt_goldEarnedPerTime,Red_Spt_kda,Red_Spt_killsPerTime,Red_Spt_wardsScorePerTime,Red_Top_assistsPerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_deathsPerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_killsPerTime,Red_Top_wardsScorePerTime
0,107405837336900396,0.001760,0.190982,0.108972,0.000785,5.083597,4.055000,0.001416,0.011231,0.002439,0.101188,0.064254,0.001017,3.921759,3.500000,0.000857,0.010386,0.003044,0.235503,0.137477,0.001924,6.370547,3.503333,0.001289,0.012620,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907,0.001678,0.159966,0.093797,0.000959,4.510053,2.753333,0.000868,0.008966,0.002355,0.269361,0.148442,0.001451,7.214419,5.043333,0.002575,0.017037,0.003898,0.189871,0.093215,0.002278,5.565174,4.167500,0.001402,0.018396,0.002757,0.277862,0.142249,0.001102,6.796912,5.183333,0.001768,0.014921,0.004666,0.064852,0.020747,0.002148,3.701635,4.166905,0.000178,0.050850,0.002332,0.270129,0.132205,0.001373,6.763587,4.088333,0.001874,0.012560
1,107405837336900398,0.002414,0.262172,0.140865,0.001073,6.741225,5.573333,0.001813,0.016469,0.002598,0.153160,0.092617,0.002151,5.444420,2.578333,0.001104,0.019234,0.001419,0.311129,0.140421,0.001744,6.214345,2.100000,0.000889,0.013025,0.004053,0.071289,0.014887,0.001316,3.885825,4.308333,0.000173,0.040550,0.002639,0.225268,0.133906,0.001379,6.417102,3.640000,0.001313,0.013368,0.001511,0.254212,0.149685,0.001490,6.659339,2.555556,0.001860,0.017417,0.002363,0.129096,0.092529,0.001810,4.954212,1.600000,0.000323,0.014165,0.002973,0.256616,0.140021,0.001747,6.468964,3.668333,0.001401,0.012788,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907,0.001361,0.263316,0.144212,0.001490,6.685741,2.194444,0.001669,0.012392
2,107405837336900400,0.002275,0.281950,0.151528,0.000851,6.701091,6.770833,0.001471,0.017603,0.002093,0.143390,0.088655,0.001600,5.184040,2.006250,0.000906,0.016029,0.002556,0.270339,0.139476,0.001277,6.420241,5.318333,0.001352,0.013276,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907,0.001606,0.223520,0.130279,0.001553,6.303885,2.154167,0.001244,0.012013,0.004352,0.230636,0.150842,0.001304,7.483906,8.641667,0.002406,0.015398,0.007017,0.156523,0.094246,0.001576,6.095752,6.220833,0.001656,0.018303,0.004793,0.272409,0.153363,0.000837,7.355654,11.531250,0.002945,0.012439,0.006931,0.071367,0.016276,0.001

In [6]:
formvalue_pivot = formvalue_pivot.merge(teamdata_df, on = "gameId", how = "left")
formvalue_pivot = formvalue_pivot.merge(headtohead_and_winner_df, on = "gameId", how = "left")
formvalue_pivot.head()

,gameId,Blue_Adc_assistsPerTime,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_deathsPerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_killsPerTime,Blue_Adc_wardsScorePerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,Blue_Jgl_killsPerTime,Blue_Jgl_wardsScorePerTime,Blue_Mid_assistsPerTime,Blue_Mid_championDamageShare,Blue_Mid_creepScorePerTime,Blue_Mid_deathsPerTime,Blue_Mid_goldEarnedPerTime,Blue_Mid_kda,Blue_Mid_killsPerTime,Blue_Mid_wardsScorePerTime,Blue_Spt_assistsPerTime,Blue_Spt_championDamageShare,Blue_Spt_creepScorePerTime,Blue_Spt_deathsPerTime,Blue_Spt_goldEarnedPerTime,Blue_Spt_kda,Blue_Spt_killsPerTime,Blue_Spt_wardsScorePerTime,Blue_Top_assistsPerTime,Blue_Top_championDamageShare,Blue_Top_creepScorePerTime,Blue_Top_deathsPerTime,Blue_Top_goldEarnedPerTime,Blue_Top_kda,Blue_Top_killsPerTime,Blue_Top_wardsScorePerTime,Red_Adc_assistsPerTime,Red_Adc_championDamageShare,Red_Adc_creepScorePerTime,Red_Adc_deathsPerTime,Red_Adc_goldEarnedPerTime,Red_Adc_kda,Red_Adc_killsPerTime,Red_Adc_wardsScorePerTime,Red_Jgl_assistsPerTime,Red_Jgl_championDamageShare,Red_Jgl_creepScorePerTime,Red_Jgl_deathsPerTime,Red_Jgl_goldEarnedPerTime,Red_Jgl_kda,Red_Jgl_killsPerTime,Red_Jgl_wardsScorePerTime,Red_Mid_assistsPerTime,Red_Mid_championDamageShare,Red_Mid_creepScorePerTime,Red_Mid_deathsPerTime,Red_Mid_goldEarnedPerTime,Red_Mid_kda,Red_Mid_killsPerTime,Red_Mid_wardsScorePerTime,Red_Spt_assistsPerTime,Red_Spt_championDamageShare,Red_Spt_creepScorePerTime,Red_Spt_deathsPerTime,Red_Spt_goldEarnedPerTime,Red_Spt_kda,Red_Spt_killsPerTime,Red_Spt_wardsScorePerTime,Red_Top_assistsPerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_deathsPerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_killsPerTime,Red_Top_wardsScorePerTime,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff,headtoHeadWinrate,headtoHeadGoldDiff,headtoHeadKillDiff,winner
0,107405837336900396,0.001760,0.190982,0.108972,0.000785,5.083597,4.055000,0.001416,0.011231,0.002439,0.101188,0.064254,0.001017,3.921759,3.500000,0.000857,0.010386,0.003044,0.235503,0.137477,0.001924,6.370547,3.503333,0.001289,0.012620,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907,0.001678,0.159966,0.093797,0.000959,4.510053,2.753333,0.000868,0.008966,0.002355,0.269361,0.148442,0.001451,7.214419,5.043333,0.002575,0.017037,0.003898,0.189871,0.093215,0.002278,5.565174,4.167500,0.001402,0.018396,0.002757,0.277862,0.142249,0.001102,6.796912,5.183333,0.001768,0.014921,0.004666,0.064852,0.020747,0.002148,3.701635,4.166905,0.000178,0.050850,0.002332,0.270129,0.132205,0.001373,6.763587,4.088333,0.001874,0.012560,0.565217,0.214286,0.208655,-3.059317,0.000120,-0.002194,0.666667,10.727451,0.009804,Red
1,107405837336900398,0.002414,0.262172,0.140865,0.001073,6.741225,5.573333,0.001813,0.016469,0.002598,0.153160,0.092617,0.002151,5.444420,2.578333,0.001104,0.019234,0.001419,0.311129,0.140421,0.001744,6.214345,2.100000,0.000889,0.013025,0.004053,0.071289,0.014887,0.001316,3.885825,4.308333,0.000173,0.040550,0.002639,0.225268,0.133906,0.001379,6.417102,3.640000,0.001313,0.013368,0.001511,0.254212,0.149685,0.001490,6.659339,2.555556,0.001860,0.017417,0.002363,0.129096,0.092529,0.001810,4.954212,1.600000,0.000323,0.014165,0.002973,0.256616,0.140021,0.001747,6.468964,3.668333,0.001401,0.012788,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907,0.001361,0.263316,0.144212,0.001490,6.685741,2.194444,0.001669,0.012392,0.272727,0.520000,-2.828609,-0.020675,-0.002835,-0.000087,0.500000,-1.099028,-0.001224,Blue
2,107405837336900400,0.002275,0.281950,0.151528,0.000851,6.701091,6.770833,0.001471,0.017603,0.002093,0.143390,0.088655,0.001600,5.184040,2.006250,0.000906,0.016029,0.002556,0.270339,0.139476,0.001277,6.420241,5.318333,0.001352,0.013276,0.003136,0.052236,0.012857,0.001036,2.802296,3.383333,0.000208,0.030907

In [7]:
formvalue_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6884 entries, 0 to 6883
Data columns (total 91 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   gameId                        6884 non-null   object 
 1   Blue_Adc_assistsPerTime       6884 non-null   float64
 2   Blue_Adc_championDamageShare  6884 non-null   float64
 3   Blue_Adc_creepScorePerTime    6884 non-null   float64
 4   Blue_Adc_deathsPerTime        6884 non-null   float64
 5   Blue_Adc_goldEarnedPerTime    6884 non-null   float64
 6   Blue_Adc_kda                  6884 non-null   float64
 7   Blue_Adc_killsPerTime         6884 non-null   float64
 8   Blue_Adc_wardsScorePerTime    6884 non-null   float64
 9   Blue_Jgl_assistsPerTime       6884 non-null   float64
 10  Blue_Jgl_championDamageShare  6884 non-null   float64
 11  Blue_Jgl_creepScorePerTime    6884 non-null   float64
 12  Blue_Jgl_deathsPerTime        6884 non-null   float64
 13  Blu

In [8]:
formvalue_pivot.describe()

,Blue_Adc_assistsPerTime,Blue_Adc_championDamageShare,Blue_Adc_creepScorePerTime,Blue_Adc_deathsPerTime,Blue_Adc_goldEarnedPerTime,Blue_Adc_kda,Blue_Adc_killsPerTime,Blue_Adc_wardsScorePerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_championDamageShare,Blue_Jgl_creepScorePerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_goldEarnedPerTime,Blue_Jgl_kda,Blue_Jgl_killsPerTime,Blue_Jgl_wardsScorePerTime,Blue_Mid_assistsPerTime,Blue_Mid_championDamageShare,Blue_Mid_creepScorePerTime,Blue_Mid_deathsPerTime,Blue_Mid_goldEarnedPerTime,Blue_Mid_kda,Blue_Mid_killsPerTime,Blue_Mid_wardsScorePerTime,Blue_Spt_assistsPerTime,Blue_Spt_championDamageShare,Blue_Spt_creepScorePerTime,Blue_Spt_deathsPerTime,Blue_Spt_goldEarnedPerTime,Blue_Spt_kda,Blue_Spt_killsPerTime,Blue_Spt_wardsScorePerTime,Blue_Top_assistsPerTime,Blue_Top_championDamageShare,Blue_Top_creepScorePerTime,Blue_Top_deathsPerTime,Blue_Top_goldEarnedPerTime,Blue_Top_kda,Blue_Top_killsPerTime,Blue_Top_wardsScorePerTime,Red_Adc_assistsPerTime,Red_Adc_championDamageShare,Red_Adc_creepScorePerTime,Red_Adc_deathsPerTime,Red_Adc_goldEarnedPerTime,Red_Adc_kda,Red_Adc_killsPerTime,Red_Adc_wardsScorePerTime,Red_Jgl_assistsPerTime,Red_Jgl_championDamageShare,Red_Jgl_creepScorePerTime,Red_Jgl_deathsPerTime,Red_Jgl_goldEarnedPerTime,Red_Jgl_kda,Red_Jgl_killsPerTime,Red_Jgl_wardsScorePerTime,Red_Mid_assistsPerTime,Red_Mid_championDamageShare,Red_Mid_creepScorePerTime,Red_Mid_deathsPerTime,Red_Mid_goldEarnedPerTime,Red_Mid_kda,Red_Mid_killsPerTime,Red_Mid_wardsScorePerTime,Red_Spt_assistsPerTime,Red_Spt_championDamageShare,Red_Spt_creepScorePerTime,Red_Spt_deathsPerTime,Red_Spt_goldEarnedPerTime,Red_Spt_kda,Red_Spt_killsPerTime,Red_Spt_wardsScorePerTime,Red_Top_assistsPerTime,Red_Top_championDamageShare,Red_Top_creepScorePerTime,Red_Top_deathsPerTime,Red_Top_goldEarnedPerTime,Red_Top_kda,Red_Top_killsPerTime,Red_Top_wardsScorePerTime,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff,headtoHeadWinrate,headtoHeadGoldDiff,headtoHeadKillDiff
count,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000
mean,0.002583,0.273469,0.152578,0.001183,7.237767,6.123678,0.002144,0.016114,0.003583,0.143597,0.090802,0.001481,5.567591,5.419990,0.001311,0.014675,0.002895,0.264537,0.142233,0.001266,6.691475,5.727651,0.001728,0.012694,0.004608,0.080800,0.019344,0.001498,4.037394,5.501715,0.000404,0.042117,0.002445,0.229265,0.133135,0.001397,6.427004,4.371858,0.001331,0.012348,0.002601,0.272960,0.152620,0.001192,7.253489,6.169173,0.002179,0.016094,0.003625,0.144006,0.090762,0.001486,5.576145,5.483362,0.001321,0.014702,0.002926,0.264564,0.142304,0.001273,6.696432,5.755534,0.001731,0.012705,0.004643,0.080889,0.019326,0.001496,4.041651,5.547620,0.000411,0.042084,0.002490,0.228543,0.133004,0.001390,6.430850,4.450222,0.001338,0.012342,0.512732,0.510731,0.182198,0.128742,0.000171,0.000110,0.495341,-0.041043,-0.000037
std,0.000793,0.036026,0.015589,0.000420,0.657719,2.346024,0.000725,0.003804,0.001082,0.024459,0.009693

In [9]:
formvalue_pivot = pitcheranalyze(formvalue_pivot)

formvalue_pivot.head()

,gameId,Blue_Adc_assistsPerTime,Blue_Adc_deathsPerTime,Blue_Adc_killsPerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_killsPerTime,Blue_Mid_assistsPerTime,Blue_Mid_deathsPerTime,Blue_Mid_killsPerTime,Blue_Spt_assistsPerTime,Blue_Top_assistsPerTime,Blue_Top_deathsPerTime,Blue_Top_killsPerTime,Red_Adc_assistsPerTime,Red_Adc_deathsPerTime,Red_Adc_killsPerTime,Red_Jgl_assistsPerTime,Red_Jgl_deathsPerTime,Red_Jgl_killsPerTime,Red_Mid_assistsPerTime,Red_Mid_deathsPerTime,Red_Mid_killsPerTime,Red_Spt_assistsPerTime,Red_Top_assistsPerTime,Red_Top_deathsPerTime,Red_Top_killsPerTime,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff,headtoHeadWinrate,headtoHeadGoldDiff,headtoHeadKillDiff,winner,Top_kdaDiff,Top_killsPerTimeDiff,Top_deathsPerTimeDiff,Top_assistsPerTimeDiff,Top_creepScorePerTimeDiff,Top_goldEarnedPerTimeDiff,Jgl_kdaDiff,Jgl_killsPerTimeDiff,Jgl_deathsPerTimeDiff,Jgl_assistsPerTimeDiff,Jgl_creepScorePerTimeDiff,Jgl_goldEarnedPerTimeDiff,Mid_kdaDiff,Mid_killsPerTimeDiff,Mid_deathsPerTimeDiff,Mid_assistsPerTimeDiff,Mid_creepScorePerTimeDiff,Mid_goldEarnedPerTimeDiff,Adc_kdaDiff,Adc_killsPerTimeDiff,Adc_deathsPerTimeDiff,Adc_assistsPerTimeDiff,Adc_creepScorePerTimeDiff,Adc_goldEarnedPerTimeDiff,Spt_assistsPerTimeDiff,Top_wardsScorePerTimeDiff,Jgl_wardsScorePerTimeDiff,Mid_wardsScorePerTimeDiff,Adc_wardsScorePerTimeDiff,Spt_wardsScorePerTimeDiff,teamWinrateDiff,teamGoldDiff,teamKillDiff
0,107405837336900396,0.001760,0.000785,0.001416,0.002439,0.001017,0.000857,0.003044,0.001924,0.001289,0.003136,0.001678,0.000959,0.000868,0.002355,0.001451,0.002575,0.003898,0.002278,0.001402,0.002757,0.001102,0.001768,0.004666,0.002332,0.001373,0.001874,0.565217,0.214286,0.208655,-3.059317,0.000120,-0.002194,0.666667,10.727451,0.009804,Red,-1.335000,-0.001006,-0.000414,-0.000654,-0.038407,-2.253534,-0.667500,-0.000544,-0.001261,-0.001460,-0.028961,-1.643415,-1.680000,-0.000478,0.000822,0.000287,-0.004772,-0.426365,-0.988333,-0.001159,-0.000666,-0.000595,-0.039471,-2.130822,-0.001530,-0.003594,-0.008010,-0.002301,-0.005806,-0.019943,0.350932,3.267973,0.002314
1,107405837336900398,0.002414,0.001073,0.001813,0.002598,0.002151,0.001104,0.001419,0.001744,0.000889,0.004053,0.002639,0.001379,0.001313,0.001511,0.001490,0.001860,0.002363,0.001810,0.000323,0.002973,0.001747,0.001401,0.003136,0.001361,0.001490,0.001669,0.272727,0.520000,-2.828609,-0.020675,-0.002835,-0.000087,0.500000,-1.099028,-0.001224,Blue,1.445556,-0.000356,-0.000111,0.001278,-0.010306,-0.268639,0.978333,0.000780,0.000341,0.000235,0.000088,0.490208,-1.568333,-0.000512,-0.000003,-0.001554,0.000400,-0.254619,3.017778,-0.000047,-0.000417,0.000903,-0.008820,0.081886,0.000918,0.000976,0.005070,0.000237,-0.000947,0.009643,-0.247273,-2.807935,-0.002748
2,107405837336900400,0.002275,0.000851,0.001471,0.002093,0.001600,0.000906,0.002556,0.001277,0.001352,0.003136,0.001606,0.001553,0.001244,0.004352,0.001304,0.002406,0.007017,0.001576,0.001656,0.004793,0.000837,0.002945,0.006931,0.003255,0.002166,0.002449,0.500000,0.518519,-0.298304,0.266528,-0.000239,0.000405,0.500000,1.640834,0.001811,Red,-3.454167,-0.001204,-0.000613,-0.001649,-0.007125,-0.583221,-4.214583,-0.000750,0.000023,-0.004924,-0.005590,-0.911711,-6.212917,-0.001593,0.000440,-0.002237,-0.013887,-0.935413,-1.870833,-0.000934,-0.000453,-0.002077,0.000687,-0.782814,-0.003795,-0.001793,-0.002274,0.000837,0.002205,-0.008071,-0.018519,-0.564832,-0.000644
3,107405837336900402,0.002019,0.001704,0.001649,0.002172,0.001521,0.001090,0.004258,0.001428,0.002076,0.003550,0.003216,0.001975,0.002777,0.001472,0.001706,0.001203,0.001957,0.001922,0.000485,0.002994,0.001874,0.001289,0.003136,0.000719,0.001706,0.001975,0.583333,0.541667,0.719165,0.063835,0.000336,-0.000091,0.500000,-0.099809,-0.000475,Blue,1.931667,0.000802,0.000269,0.002498,-0.020408,0.157579,0.808333,0.000605,-0.000401,0.000215,0.000615,0.504056,2.398333,0.000787,-0.000445,0.001265,-0.006775,0.340173,1.566667,0.000446,-0.000002,0.000547,0.0131

In [10]:
formvalue_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6884 entries, 0 to 6883
Data columns (total 70 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   gameId                     6884 non-null   object 
 1   Blue_Adc_assistsPerTime    6884 non-null   float64
 2   Blue_Adc_deathsPerTime     6884 non-null   float64
 3   Blue_Adc_killsPerTime      6884 non-null   float64
 4   Blue_Jgl_assistsPerTime    6884 non-null   float64
 5   Blue_Jgl_deathsPerTime     6884 non-null   float64
 6   Blue_Jgl_killsPerTime      6884 non-null   float64
 7   Blue_Mid_assistsPerTime    6884 non-null   float64
 8   Blue_Mid_deathsPerTime     6884 non-null   float64
 9   Blue_Mid_killsPerTime      6884 non-null   float64
 10  Blue_Spt_assistsPerTime    6884 non-null   float64
 11  Blue_Top_assistsPerTime    6884 non-null   float64
 12  Blue_Top_deathsPerTime     6884 non-null   float64
 13  Blue_Top_killsPerTime      6884 non-null   float

In [11]:
formvalue_pivot.describe()

,Blue_Adc_assistsPerTime,Blue_Adc_deathsPerTime,Blue_Adc_killsPerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_killsPerTime,Blue_Mid_assistsPerTime,Blue_Mid_deathsPerTime,Blue_Mid_killsPerTime,Blue_Spt_assistsPerTime,Blue_Top_assistsPerTime,Blue_Top_deathsPerTime,Blue_Top_killsPerTime,Red_Adc_assistsPerTime,Red_Adc_deathsPerTime,Red_Adc_killsPerTime,Red_Jgl_assistsPerTime,Red_Jgl_deathsPerTime,Red_Jgl_killsPerTime,Red_Mid_assistsPerTime,Red_Mid_deathsPerTime,Red_Mid_killsPerTime,Red_Spt_assistsPerTime,Red_Top_assistsPerTime,Red_Top_deathsPerTime,Red_Top_killsPerTime,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff,headtoHeadWinrate,headtoHeadGoldDiff,headtoHeadKillDiff,Top_kdaDiff,Top_killsPerTimeDiff,Top_deathsPerTimeDiff,Top_assistsPerTimeDiff,Top_creepScorePerTimeDiff,Top_goldEarnedPerTimeDiff,Jgl_kdaDiff,Jgl_killsPerTimeDiff,Jgl_deathsPerTimeDiff,Jgl_assistsPerTimeDiff,Jgl_creepScorePerTimeDiff,Jgl_goldEarnedPerTimeDiff,Mid_kdaDiff,Mid_killsPerTimeDiff,Mid_deathsPerTimeDiff,Mid_assistsPerTimeDiff,Mid_creepScorePerTimeDiff,Mid_goldEarnedPerTimeDiff,Adc_kdaDiff,Adc_killsPerTimeDiff,Adc_deathsPerTimeDiff,Adc_assistsPerTimeDiff,Adc_creepScorePerTimeDiff,Adc_goldEarnedPerTimeDiff,Spt_assistsPerTimeDiff,Top_wardsScorePerTimeDiff,Jgl_wardsScorePerTimeDiff,Mid_wardsScorePerTimeDiff,Adc_wardsScorePerTimeDiff,Spt_wardsScorePerTimeDiff,teamWinrateDiff,teamGoldDiff,teamKillDiff
count,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000
mean,0.002583,0.001183,0.002144,0.003583,0.001481,0.001311,0.002895,0.001266,0.001728,0.004608,0.002445,0.001397,0.001331,0.002601,0.001192,0.002179,0.003625,0.001486,0.001321,0.002926,0.001273,0.001731,0.004643,0.002490,0.001390,0.001338,0.512732,0.510731,0.182198,0.128742,0.000171,0.000110,0.495341,-0.041043,-0.000037,-0.078364,-0.000006,0.000007,-0.000044,0.000131,-0.003846,-0.063372,-0.000010,-0.000005,-0.000042,0.000040,-0.008554,-0.027883,-0.000003,-0.000007,-0.000031,-0.000071,-0.004957,-0.045495,-0.000035,-0.000009,-0.000019,-0.000042,-0.015721,-0.000035,0.000006,-0.000027,-0.000011,0.000020,0.000033,0.002002,0.053455,0.000061
std,0.000793,0.000420,0.000725,0.001082,0.000490,0.000532,0.000884,0.000433,0.000637,0.001349,0.000794,0.000441,0.000514,0.000795,0.000434,0.000744,0.001091,0.000505,0.000527,0.000909,0.000448,0.000625,0.001352,0.000797,0.000440,0.000515,0.147692,0.149821,2.430872,2.479695,0.002330,0.002414,0.181706,3.745208,0.003747,2.509211,0.000679,0.000592,0.001032,0.014589,0.724985,3.100610,0.000696,0.000653,0.001348,0.011894,0.648624,3.189385,0.000808,0.000576,0.001140,0.015021,0.750852,3.261740,0.000952,0.000573,0.001017,0.017757,0.809322,0.001704,0.002540,0.003173,0.002866,0.003891,0.007543,0.203664,3.402129,0.003310
min,0.000000,0.000000,0.000000,0.000287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000317,0.000000,0.000000,0.000379,0.000000,0.000000,0.000000,0.000000,0.000000,0.000162,0.000189,0.000000,0.000000,0.040000,0.041667,-12.985075,-13.412218,-0.013292,-0.015076,0.076923,-13.614894,-0.019380,-10.215000,-0.002727,-0.002847,-0.004921,-0.102547,-3.938532,-12.260000,-0.004715,-0.003784,-0.006084,-0.066410,-3.763295,-12.911667,-0.004175,-0.002859,

In [12]:
suffix_list = ["assistsPerTime", "championDamageShare", "creepScorePerTime", "deathsPerTime", "goldEarnedPerTime", "kda", "killsPerTime", "wardsScorePerTime", "Winrate", "GoldDiff", "KillDiff",
               "wardsScorePerTimeDiff", "assistsPerTimeDiff", "kdaDiff", "killsPerTimeDiff", "deathsPerTimeDiff", "creepScorePerTimeDiff", "goldEarnedPerTimeDiff"]
scaler_dict = {}
for suffix in suffix_list:
    filtered_columns = [col for col in formvalue_pivot.columns if col.split('_')[-1] in suffix]
    if len(filtered_columns) != 0:
        filtered_df = formvalue_pivot[filtered_columns]
        scaler_dict[suffix] = StandardScaler()
        data_have_common_suffix_list = []
        for column in filtered_df.columns:
            data_have_common_suffix_list.extend(filtered_df[column].values)
        data_have_common_suffix_list = np.array(data_have_common_suffix_list).reshape(-1,1)
        scaler_dict[suffix] = scaler_dict[suffix].fit(data_have_common_suffix_list)             # 모든 공통 suffix에 대해 한번 fit하고
        for column in filtered_df.columns:
            formvalue_pivot[column] = scaler_dict[suffix].transform(formvalue_pivot[column].values.reshape(-1, 1))  # 각자 transform.
scaler_dict["headtoHeadWinrate"] = StandardScaler()
formvalue_pivot["headtoHeadWinrate"] = scaler_dict["headtoHeadWinrate"].fit_transform(formvalue_pivot["headtoHeadWinrate"].values.reshape(-1, 1))
scaler_dict["headtoHeadGoldDiff"] = StandardScaler()
formvalue_pivot["headtoHeadGoldDiff"] = scaler_dict["headtoHeadGoldDiff"].fit_transform(formvalue_pivot["headtoHeadGoldDiff"].values.reshape(-1, 1))
scaler_dict["headtoHeadKillDiff"] = StandardScaler()
formvalue_pivot["headtoHeadKillDiff"] = scaler_dict["headtoHeadKillDiff"].fit_transform(formvalue_pivot["headtoHeadKillDiff"].values.reshape(-1, 1))
scaler_dict["teamWinrateDiff"] = StandardScaler()
formvalue_pivot["teamWinrateDiff"] = scaler_dict["teamWinrateDiff"].fit_transform(formvalue_pivot["teamWinrateDiff"].values.reshape(-1, 1))
scaler_dict["teamGoldDiff"] = StandardScaler()
formvalue_pivot["teamGoldDiff"] = scaler_dict["teamGoldDiff"].fit_transform(formvalue_pivot["teamGoldDiff"].values.reshape(-1, 1))
scaler_dict["teamKillDiff"] = StandardScaler()
formvalue_pivot["teamKillDiff"] = scaler_dict["teamKillDiff"].fit_transform(formvalue_pivot["teamKillDiff"].values.reshape(-1, 1))
formvalue_pivot.describe()

,Blue_Adc_assistsPerTime,Blue_Adc_deathsPerTime,Blue_Adc_killsPerTime,Blue_Jgl_assistsPerTime,Blue_Jgl_deathsPerTime,Blue_Jgl_killsPerTime,Blue_Mid_assistsPerTime,Blue_Mid_deathsPerTime,Blue_Mid_killsPerTime,Blue_Spt_assistsPerTime,Blue_Top_assistsPerTime,Blue_Top_deathsPerTime,Blue_Top_killsPerTime,Red_Adc_assistsPerTime,Red_Adc_deathsPerTime,Red_Adc_killsPerTime,Red_Jgl_assistsPerTime,Red_Jgl_deathsPerTime,Red_Jgl_killsPerTime,Red_Mid_assistsPerTime,Red_Mid_deathsPerTime,Red_Mid_killsPerTime,Red_Spt_assistsPerTime,Red_Top_assistsPerTime,Red_Top_deathsPerTime,Red_Top_killsPerTime,Blue_Winrate,Red_Winrate,Blue_GoldDiff,Red_GoldDiff,Blue_KillDiff,Red_KillDiff,headtoHeadWinrate,headtoHeadGoldDiff,headtoHeadKillDiff,Top_kdaDiff,Top_killsPerTimeDiff,Top_deathsPerTimeDiff,Top_assistsPerTimeDiff,Top_creepScorePerTimeDiff,Top_goldEarnedPerTimeDiff,Jgl_kdaDiff,Jgl_killsPerTimeDiff,Jgl_deathsPerTimeDiff,Jgl_assistsPerTimeDiff,Jgl_creepScorePerTimeDiff,Jgl_goldEarnedPerTimeDiff,Mid_kdaDiff,Mid_killsPerTimeDiff,Mid_deathsPerTimeDiff,Mid_assistsPerTimeDiff,Mid_creepScorePerTimeDiff,Mid_goldEarnedPerTimeDiff,Adc_kdaDiff,Adc_killsPerTimeDiff,Adc_deathsPerTimeDiff,Adc_assistsPerTimeDiff,Adc_creepScorePerTimeDiff,Adc_goldEarnedPerTimeDiff,Spt_assistsPerTimeDiff,Top_wardsScorePerTimeDiff,Jgl_wardsScorePerTimeDiff,Mid_wardsScorePerTimeDiff,Adc_wardsScorePerTimeDiff,Spt_wardsScorePerTimeDiff,teamWinrateDiff,teamGoldDiff,teamKillDiff
count,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6.884000e+03,6.884000e+03,6.884000e+03,6884.000000,6.884000e+03,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6884.000000,6.884000e+03,6.884000e+03,6.884000e+03
mean,-0.626434,-0.394958,0.889633,0.327401,0.388140,-0.566866,-0.328850,-0.177496,0.162073,1.304152,-0.757503,0.165824,-0.531795,-0.608790,-0.371788,0.950747,0.367155,0.401513,-0.549835,-0.299325,-0.159893,0.166566,1.337819,-0.715487,0.148670,-0.520480,0.006728,-0.006728,0.010885,-0.010885,0.012825,-0.012825,1.321172e-16,2.064331e-17,8.257324e-18,-0.008115,-2.440825e-06,0.000009,-0.000040,0.007802,0.006013,-0.003166,-0.000006,-0.000005,-0.000037,0.001701,-0.000387,0.008547,0.000002,-0.000007,-0.000024,-0.005703,0.004503,0.002734,-0.000037,-0.000009,-0.000009,-0.003800,-0.010129,-0.000029,0.000442,-0.007143,-0.003457,0.003539,0.006620,8.257324e-18,-1.238599e-17,-1.238599e-17
std,0.755780,1.103996,1.268400,1.031866,1.287807,0.930054,0.843047,1.136457,1.114657,1.286657,0.756676,1.159156,0.898877,0.757774,1.140267,1.301868,1.040093,1.326058,0.921703,0.866643,1.175644,1.093440,1.289490,0.760035,1.155715,0.900159,0.992868,1.007181,0.990022,1.009906,0.982249,1.017422,1.000073e+00,1.000073e+00,1.000073e+00,0.828195,8.315486e-04,0.000725,0.001264,0.975219,0.985469,1.023393,0.000853,0.000800,0.001651,0.795058,0.881673,1.052695,0.000989,0.000706,0.001396,1.004131,1.020631,1.076576,0.001166,0.000702,0.001246,1.187023,1.100108,0.002086,0.577407,0.721279,0.651520,0.884521,1.714675,1.000073e+00,1.000073e+00,1.000073e+00
min,-3.089099,-3.500985,-2.860579,-2.815120,-3.500985,-2.860579,-3.089099,-3.500985,-2.860579,-3.089099,-3.089099,-3.500985,-2.860579,-2.786417,-3.500985,-2.860579,-2.727944,-3.500985,-2.860579,-3.089099,-3.500985,-2.860579,-2.934819,-2.908522,-3.500985,-2.860579,-3.171250,-3.160045,-5.351753,-5.525716,-5.662345,-6.414518,-2.302890e+00,-3.624588e+00,-5.162957e+00,-3.353833,-3.333930e-03,-0.003486,-0.006

In [13]:
joblib.dump((formvalue_pivot, scaler_dict), "../data/dataset_draft5_7.pkl")

['../data/dataset_draft5_7.pkl']

In [14]:
#formvalue_pivot.to_excel("../data/dataset_draft5_4.xlsx")